In [12]:
%matplotlib notebook
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve
from sklearn.feature_selection import SelectKBest, RFECV, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from functools import reduce
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import make_scorer
#from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter
import tensorflow as tf
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.metrics import accuracy

In [3]:
data=pd.read_csv(filepath_or_buffer='generated_csv/hrv_completed.csv')
pd.set_option('display.max_columns', None)
    
data.participant=data.participant.astype('category').cat.codes #turn the participant column into categorical numbers (instead of keep the string with letters+numbers because it would raise an error later on)
scaler=MinMaxScaler(feature_range=(1, 3))
cols_to_normalize=['arousal','arousal_groups','valence','valence_groups','arousal_valence_4_grouping','wasted','wasted_groups','location','ts','activity','hr_avg','hr_min','hr_max','hr_std','rmssd','sdrr','delta_time_min','hrv_baseline','rmssd_compared_hrv_baseline'] #normalize all columns

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True): # returns a Pandas DataFrame of series framed for supervised learning
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)

	cols, names = list(), list()
	#columns=['participant','arousal','arousal_groups','valence','valence_groups','arousal_valence_4_grouping','wasted','wasted_groups','location','hour_min','ts','activity','hr_avg','hr_min','hr_max','hr_std','rmssd','sdrr','delta_time_min']

	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		"""shift() -> this function is used to create copies of columns that are pushed forward (rows of NaN values added to the front) or pulled back (rows of NaN values added to the end). This is the behavior required to create
		columns of lag observations as well as columns of forecast observations for a time series dataset in a supervised learning format."""
		names += [('var%s(t-%d)' % (j, i)) for j in range(n_vars)]
	
    # forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%s(t)' % (j)) for j in range(n_vars)]
		else:
			names += [('var%s(t+%d)' % (j, i)) for j in range(n_vars)]
	
    # put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	
    # drop rows with NaN values
	#if dropnan:
	#	agg.dropna(inplace=True)
	return agg

With Cross validation

In [13]:
results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'w')
results.write('Classifier,Avg_AUC,Avg_Acc,Number_of_Features,Features,AUC_each_fold,Acc_each_fold,TPR_multiclass_each_fold,FPR_multiclass_each_fold,Best_Hyperparameters\n') #columns names
results.close()

#predicted_real_labels = open('Best_scenarios/sensor/classifiers_standardization/predicted_HRV_real_labels.csv', 'w')
#predicted_real_labels.close()

KNN1,DT1,RF1,NN1,AB1,NB1,QDA1,SVM1,number_of_features1=[],[],[],[],[],[],[],[],[]

#Decision tree does not need normalization
data_copy=data.copy()
data_copy[cols_to_normalize]=scaler.fit_transform(data[cols_to_normalize])
values=data_copy.values
#values=data.values

# convert series to supervised learning
reframed = series_to_supervised(values, 1, 1) #shift to get the time features

"""
Reframed index meaning:
0: participant(t-1); 1:arousal(t-1); 2:arousal_groups(t-1); 3:valence(t-1); 4:valence_groups(t-1); 5: arousal_valence_4_grouping(t-1); 6: wasted(t-1); 7: wasted_groups(t-1); 8: location(t-1) 
9: hour_min(t-1); 10: ts(t-1); 11: activity(t-1); 12: hr_avg(t-1); 13: hr_min(t-1); 14: hr_max(t-1); 15: hr_std(t-1); 16: rmssd(t-1); 17: sdrr(t-1); 18: delta_time_min(t-1); 19: hrv_baseline(t-1); 20: rmssd_compared_hrv_baseline(t-1)
21: participant(t); 22:arousal(t); 23:arousal_groups(t); 24:valence(t); 25:valence_groups(t); 26: arousal_valence_4_grouping(t); 27: wasted(t); 28: wasted_groups(t); 29: location(t) 30: hour_min(t); 31: ts(t); 
32: activity(t); 33: hr_avg(t); 34: hr_min(t); 35: hr_max(t); 36: hr_std(t); 37: rmssd(t); 38: sdrr(t); 39: delta_time_min(t); 40: hrv_baseline(t); 41: rmssd_compared_hrv_baseline(t)
"""

reframed.drop(reframed.columns[[1,2,3,4,5,6,7,8,9,11,18,22,23,24,26,27,28,29,30,32,40]], axis=1, inplace=True) #18 -> delta_time_min(t-1); 24 -> valence(t); 26 -> arousal_valence_4_grouping.(t) --> makes no sense to keep; 9:hour_min(t-1) + 30 hour_min(t-1) -> we already have ts; it won't add new information

"""
reframed index meaning after the drop:
0: participant(t-1); 1 ts(t-1); 2: hr_avg(t-1); 3: hr_min(t-1); 4: hr_max(t-1); 5: hr_std(t-1); 6: rmssd(t-1); 7: sdrr(t-1); 8: hrv_baseline(t-1); 9: rmssd_compared_hrv_baseline(t-1); 10: participant(t);
11:valence_groups(t); 12: ts(t); 13: hr_avg(t); 14: hr_min(t); 15: hr_max(t); 16: hr_std(t); 17: rmssd(t); 18: sdrr(t); 19: delta_time_min(t); 20: rmssd_compared_hrv_baseline(t)
"""
print(reframed)
#make the reframed only have rows with data from one participant (and not rows that have 2 participants data)
X=[]
y=[]
for index, row in reframed.iterrows():
    row_list=row.to_list()
    if row_list[0]==row_list[10]: #if the data belongs to the same participant
        X.append(row_list[1:10]+row_list[12:])
        y.append(row_list[11])


"""
X:
ts(t-1); hr_avg(t-1); hr_min(t-1); hr_max(t-1); hr_std(t-1); rmssd(t-1); sdrr(t-1); hrv_baseline(t-1); rmssd_compared_hrv_baseline(t-1);ts(t); hr_avg(t); hr_min(t); hr_max(t); hr_std(t); rmssd(t);
sdrr(t); delta_time_min(t); rmssd_compared_hrv_baseline(t)
"""

X=np.array(X)
y=np.array(y)
y=y.ravel()

cols=['ts(t-1)', 'hr_avg(t-1)', 'hr_min(t-1)', 'hr_max(t-1)', 'hr_std(t-1)', 'rmssd(t-1)', 'sdrr(t-1)', 'hrv_baseline(t-1)', 'rmssd_compared_hrv_baseline(t-1)', 'ts(t)', 'hr_avg(t)', 'hr_min(t)', 'hr_max(t)',
'hr_std(t)', 'rmssd(t)','sdrr(t)', 'delta_time_min(t)', 'rmssd_compared_hrv_baseline(t)']


def classifiers_definition(X_train):
    classifiers = [
        ('KNearest Neighbors', KNeighborsClassifier(), {
                'weights': ['uniform', 'distance'],
                'algorithm': ['auto']}), #'ball_tree', 'kd_tree', 'brute'

        ('Decision Tree', DecisionTreeClassifier(), {
            'criterion': ['gini', 'entropy'],
            'max_depth': [2,3,4,5],
            'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
            'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
            'max_features': list(range(1, X_train.shape[1])),
            'random_state': [0]}),

        ('Random Forest', RandomForestClassifier(), {
            'criterion': ['gini', 'entropy'],
            'max_depth': [2,3,4,5],
            'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
            'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
            'max_features': list(range(1, X_train.shape[1])),
            'random_state': [0,42]}),

        ('Neural Network', MLPClassifier(max_iter=1000), {
                'alpha': 10.0 ** -np.arange(1, 7),
                'solver': ['adam'],
                'random_state': [0,42]}),

        ('AdaBoost', AdaBoostClassifier(), {
                'algorithm': ['SAMME', 'SAMME.R'],
                'random_state': [0,42]}),

        ('Naive Bayes', GaussianNB(), {}),

        ('QDA', QuadraticDiscriminantAnalysis(), {}),
        
        ('RBF SVM', SVC(probability=True), {
                'random_state': [0,42]})
        ]

    return classifiers

def score_model(cols):
    #number_of_features=[]
    #accuracies_valence_groups_time, number_of_features_valence_groups_time, val_score_folds = [],[],[]

    cv= StratifiedKFold(n_splits=5)
    KNN_acc,DT_acc,RF_acc,NN_acc,AB_acc,NB_acc,QDA_acc,SVM_acc=[],[],[],[],[],[],[],[] #each list will hold the test scores, for each classifier, for the different amount of features.
    KNN_auc,DT_auc,RF_auc,NN_auc,AB_auc,NB_auc,QDA_auc,SVM_auc=[],[],[],[],[],[],[],[]

    # iterate through num features for feature selection
    for k in range(2, len(cols)+1):
        KNN,DT,RF,NN,AB,NB,QDA,SVM=[],[],[],[],[],[],[],[] #each of these lists will hold the validation scores in each fold (cv) for each classifier for a certain amount of features.
        number_of_features=[]

        indexes=[]
        print('--- K: {}'.format(k))
        selector = SelectKBest(f_classif, k=k)
        selector.fit(X, y)
        X_new = selector.transform(X)
        mask = selector.get_support(indices=True)
        features = ''
        for i,index in enumerate(mask):
            features += cols[index] + ';'

        #train test split -> we won't do it so we use the whole data for the cross validation
        #X_train, X_test, y_train, y_test = train_test_split(X_new, y, stratify=y, shuffle=True, test_size=.3, train_size=.7, random_state=0)

        #definition of the oversampling techniques; however, these will only really happen during each fold of the cross validation
        oversample=SMOTE()        
        fold_number=1

        for train_fold_index, test_fold_index in cv.split(X_new, y):
            print('fold_number:', fold_number)
            X_train_fold, y_train_fold = X_new[train_fold_index], y[train_fold_index]
            X_test_fold, y_test_fold = X_new[test_fold_index], y[test_fold_index]

            X_over_train, y_over_train= oversample.fit_resample(X_train_fold, y_train_fold)

            classifiers=classifiers_definition(X_over_train)

            for clf in classifiers:
                if bool(clf[2]):
                    n_iter = reduce((lambda _x, _y: _x * _y), [len(clf[2][k]) for k in clf[2]]) #number of scenarios in each algorithm
                else:
                    n_iter = 1

                print(clf[0])
                print('Grid, {}'.format(n_iter))
                
                if fold_number==1:
                    tuned_clf = RandomizedSearchCV(clf[1], clf[2], n_jobs=2, return_train_score=True) #when cv is not specified it takes the value of 5.
                
                #fit the model
                tuned_clf.fit(X_over_train, y_over_train)
                y_pred_proba=tuned_clf.predict_proba(X_test_fold)[::,1] #define metrics; to create a ROC curve we need to get the TPR and FPR
                #print('y_pred_proba',y_pred_proba)
                fpr_multiclass,tpr_multiclass,auc_multiclass=[],[],[]

                for iteration in range(3):
                    y_test_bin=[]
                    for element in y_test_fold:
                        if (iteration==0 and element==2) or (iteration==0 and element==3):
                            y_test_bin.append(0)
                        elif iteration==0 and element==1:
                            y_test_bin.append(1)
                        elif (iteration==1 and element==1) or (iteration==1 and element==3):
                            y_test_bin.append(0)
                        elif (iteration==1 and element==2):
                            y_test_bin.append(1)
                        elif (iteration==2 and element==1) or (iteration==2 and element==2):
                            y_test_bin.append(0)
                        elif (iteration==2 and element==3):
                            y_test_bin.append(1)

                    fpr,tpr,_=metrics.roc_curve(y_test_bin, y_pred_proba)
                    auc=metrics.roc_auc_score(y_test_bin, y_pred_proba)

                    fpr_multiclass.append(fpr)
                    tpr_multiclass.append(tpr)
                    auc_multiclass.append(auc)

                    if iteration==0:
                        auc_multiclass_sum=auc
                    else:
                        auc_multiclass_sum+=auc


                auc_multiclass_avg=auc_multiclass_sum/3

                print('AUC_avg:',auc_multiclass_avg)

                accuracy=accuracy_score(y_test_fold, tuned_clf.predict(X_test_fold))
                #tpr_sum,fpr_sum,auc_sum,accuracy_sum=0,0,0,0

                if clf[0]=='KNearest Neighbors':
                    KNN.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(KNN):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        KNN_acc.append(accuracy_sum/5)
                        KNN_auc.append(accuracy_sum/5)
                            
                        #KNN,KNN_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)

                elif clf[0]=='Decision Tree':
                    DT.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(DT):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        DT_acc.append(accuracy_sum/5)
                        DT_auc.append(accuracy_sum/5)
                    
                    #DT,DT_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='Random Forest':
                    RF.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(RF):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        RF_acc.append(accuracy_sum/5)
                        RF_auc.append(accuracy_sum/5)

                    #RF,RF_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='Neural Network':
                    NN.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(NN):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        NN_acc.append(accuracy_sum/5)
                        NN_auc.append(accuracy_sum/5)

                    #NN,NN_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='AdaBoost':
                    AB.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(AB):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        AB_acc.append(accuracy_sum/5)
                        AB_auc.append(accuracy_sum/5)
                    
                    #AB,AB_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='Naive Bayes':
                    NB.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(NB):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        NB_acc.append(accuracy_sum/5)
                        NB_auc.append(accuracy_sum/5)

                    #NB,NB_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='QDA':
                    QDA.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(QDA):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        QDA_acc.append(accuracy_sum/5)
                        QDA_auc.append(accuracy_sum/5)

                    #QDA,QDA_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='RBF SVM':
                    SVM.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(SVM):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('Best_scenarios/sensor/classifiers_standardization/results_HRV_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        SVM_acc.append(accuracy_sum/5)
                        SVM_auc.append(accuracy_sum/5)
                    #SVM,SVM_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)

            fold_number+=1
        number_of_features.append(k)

    return KNN_acc,DT_acc,RF_acc,NN_acc,AB_acc,NB_acc,QDA_acc,SVM_acc,KNN_auc,DT_auc,RF_auc,NN_auc,AB_auc,NB_auc,QDA_auc,SVM_auc,number_of_features

KNN_acc,DT_acc,RF_acc,NN_acc,AB_acc,NB_acc,QDA_acc,SVM_acc,KNN_auc,DT_auc,RF_auc,NN_auc,AB_auc,NB_auc,QDA_auc,SVM_auc,number_of_features=score_model(cols)

    var0(t-1) var10(t-1) var12(t-1) var13(t-1) var14(t-1) var15(t-1)  \
0         NaN        NaN        NaN        NaN        NaN        NaN   
1          70   1.889141   2.300624   2.326582   1.967033   1.051876   
2          38   1.656352   2.620214   2.663291   2.428571   1.077375   
3          38    1.78062   2.274769   2.308861   1.967033   1.053294   
4          38   1.890979   2.923327   2.875949   2.824176   1.145025   
..        ...        ...        ...        ...        ...        ...   
734        13   1.013755   2.224998   2.202532   1.967033   1.132318   
735        13   1.302085   2.141778   2.149367   1.769231   1.056088   
736        13   1.681003   1.954372   2.025112   2.406593        3.0   
737        72    2.06955   2.018858   2.025316   1.659341   1.062024   
738        72    2.13732    2.08461   1.972152   1.747253   1.180711   

    var16(t-1) var17(t-1) var19(t-1) var20(t-1) var0(t) var4(t)  var10(t)  \
0          NaN        NaN        NaN        NaN      70   

c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.5219226611011208
Decision Tree
Grid, 400
AUC_avg: 0.5262782299939507
Random Forest
Grid, 800
AUC_avg: 0.5234876821950053
Neural Network
Grid, 12
AUC_avg: 0.5297023628217151
AdaBoost
Grid, 4


c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.48590993481300665
Naive Bayes
Grid, 1
AUC_avg: 0.519219069360193
QDA
Grid, 1
AUC_avg: 0.5103873257643532
RBF SVM
Grid, 2


...

AUC_avg: 0.48185624641935326
QDA
Grid, 1


c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.48185624641935326
RBF SVM
Grid, 2


c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.48185624641935326


In [14]:
"""
from PIL import Image
number_of_features=np.arange(2,34,1)
KNN_score=np.array(KNN_acc)
DT_score=np.array(DT_acc)
RF_score=np.array(RF_acc)
NN_score=np.array(NN_acc)
AB_score=np.array(AB_acc)
NB_score=np.array(NB_acc)
QDA_score=np.array(QDA_acc)
SVM_score=np.array(SVM_acc)

plt.plot(KNN_score, color='r', label='k nearest neighbours')
plt.plot(DT_score, color='r', label='decision tree')
plt.plot(RF_score, color='b', label='random forest')
plt.plot(NN_score, color='b', label='neural network')
plt.plot(AB_score, color='m', label='adaboost')
plt.plot(NB_score, color='c', label='naive bayes')
plt.plot(QDA_score, color='y', label='qda')
plt.plot(SVM_score, color='y', label='svm')
plt.legend()
plt.show()
plt.savefig('Best_scenarios/sensor/classifiers_standardization/classifiers_features_accuracies.png')
plt.close()
"""

<IPython.core.display.Javascript object>